# Archaic to Modern Italian with Context Learning

In [1]:
# Import Datases to work with Transformers by Hugging-Face
import torch
import pandas as pd

# Imports for Transformers
from transformers import AutoTokenizer  # Datasets
from transformers import DataCollatorForSeq2Seq
import numpy as np  # Evaluation
import evaluate
from datasets import Dataset, load_dataset
import matplotlib.pyplot as plt
import os
from transformers import TrainerCallback
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, PeftModelForSeq2SeqLM # Optimize traning for big models! (more than 1B parameters)
import numpy as np
import evaluate # Assicurati che evaluate sia importato
from datasets.features import Value, Features

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Globals

In [61]:
DATASET = "dataset.csv"
FEATURES = Features(
    {
        "Author": Value(dtype="string"),
        "Date": Value(dtype="string"),
        "Region": Value(dtype="string"),
        "Sentence": Value(dtype="string")
    }
)
network = "google/mt5-base"
device = ('cuda' if torch.cuda.is_available() else "cpu")

## Import the Dataset and Tokenize


In [62]:
hf = Dataset.from_csv(DATASET, features=FEATURES).shuffle(seed=42).train_test_split(test_size=0.05, seed=42)
print(hf["train"].take(5)[:5])

{'Author': ['Giovanni da San Miniato', 'Brunetto Latini', 'Filippo da Santa Croce', "Fatti de' Romani (H+R)", 'Bono Giamboni'], 'Date': ['1415', '1294', '1323', '1313', '1292'], 'Region': ['tosc.', 'fior.', 'tosc.', 'fior.', 'fior.'], 'Sentence': ['di coloro che cominciano a fare buone operazioni, cioè che colui che ancora non sa amare il prossimo come sé medesimo già cominci a temere i giudicii di Dio. E perché', 'molto di maggiore memoria saranno faccendole al re, perciò che nella nostra cittade sempre fue santo e glorioso il nome reale, e sse furono compagni fue il loro nome santissimo;', "quello che sopra tutti gli altri perdonasse a' cittadini, e a cui più sicuramente possiate credere; poi ch'egli fu vostro comandatore.", "non contasterebe a sua virtude. Allora che il navilio si fue impinto e messo in alto mare per andare diritto per mezo questo trapasso periglioso, l'aira divenne nuvolosa e pioveginosa", "Tarentini, i quali erano nati di quegli di Lacedemonia et facta da lloro no

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(network)

In [42]:
def standard_prompt(examples):
    """
    standard_prompt function to create the input for the model
    It takes a list of examples and returns a list of strings
    where each string is a prompt for the model.
    """
    inputs = ["translate \"" + example + "\" to Italian: " for example in examples["Sentence"]]
    model_inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True)
    return model_inputs

def advanced_prompt(examples):
    #TODO: implement advanced_prompt function
    inputs = []
    for i in range(len(examples["Sentence"])):
        inputs.append(
            f"Author: {examples['Author'][i]}\nDate: {examples['Date'][i]}\nRegion: {examples['Region'][i]}\nSentence: {examples['Sentence'][i]}"
        )
    return inputs

In [43]:
standard_token = hf.map(standard_prompt, batched=True)

Map: 100%|██████████| 5/5 [00:00<00:00, 1504.63 examples/s]


In [44]:
src = standard_token["train"]['input_ids'][1]
print(tokenizer.decode(src, skip_special_tokens=True))

translate "molto di maggiore memoria saranno faccendole al re, perci che nella nostra cittade sempre fue santo e glorioso il nome reale, e sse furono compagni fue il loro nome santissimo;" to Italian:


## Model

In [46]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(network)

In [48]:
# TODO: implement the EvalModel class

model = model.eval()
text = standard_token['test'][2]["Sentence"]
inputs = tokenizer(text, return_tensors="pt")
ids = inputs.input_ids.to(device)
attention = inputs.attention_mask.to(device)

output = model.generate(input_ids=ids,  attention_mask=attention, max_new_tokens=120, do_sample=True, top_k=10, top_p=0.95)
output = tokenizer.decode(output[0], skip_special_tokens=True)

In [51]:
print(f"Original Sentence:\n {text}")
print(f"Translated Sentence:\n {output}")

Original Sentence:
 Signiore", et nonn ebbe in fastido Cristo cotali parole d'udire.
Translated Sentence:
 " "" et nonn ebbe et nonn in fastido Cristo cotali parole d'udire.
